## Game Design -  Payout Calculation for online slot game

In this project, the first game (Vegas Ca$h) in Slotomania, a popular social casino app, is used as an example, to design the game mathematics and probability for the payout calculation of the game. The same approach is applicable to any other slot game. The actual game is very complex due to various features. Therefore, some of the features have been ignored in order to simplify the calculations, however, one can incorporate additional features into the calculations once the foundational solution is developed.

In this simplified version, only the Grand jackpot win is considered for the payout calculation. The major, mini and minor wins (as featured in the original game) are ruled out of the Return to Player (RTP) calculations. 

In the base game, 6 to 14 scattered 'chip' symbols trigger the coid grab feature. In the coid grab feature, if 6 or more 'chip' symbols appear on the screen, then coid values appearing on the scattered symbols are won. Grand jackpot is awarded when 15 'Chip' symbols appear on the screen in the coid grab feature.

The 'chip' symbols that trigger the coin grab feature are held in position and all other positions turn into individual spinning reels. 3 free spins are awarded as coin grab feature is triggered. If one or more additional 'chip' appear in any position these are also held and the number of free spins remaining is reset to 3.

During the base game, 3 or more scattered bonus symbol (dollar sign) trigger the money case feature. In the money case feature, the number of free games awarded depends on the number of scattered symbols. In the following program, it is assumed that 3, 4 and 5 symbols trigger 6, 9 and 15 free games, respectively. However, the possibility of winning additional 'wild' symbols in the money grab feature (as featured in the original game) is ruled out of the calculations.

It is a 50 line game. The Jackpot prize and the bet are assumed to be 250,000 credits and 1 credit per line, respectively. It should be noted that the hits and therefore win increases linearly with additional paylines, hence the bet is similarly increased to keep the RTP constant. Therefore, it is assumed that the RTP is independent of the number of paylines in the game. 

In [1]:
%run utils.py
%reload_ext autoreload
%autoreload 2

import utils
import importlib
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\sams_chw\\PycharmProjects\\Slotomania\\utils.py'>

In [2]:
import pandas as pd
import numpy as np
from ipynb.fs.full.utils import *

reel_count = 5
bet = 1

In [3]:
df = pd.DataFrame(index=['wild', 'scatter', 'chip', 'man', 
                        'car', 'casino', 'cash', 'dice', 
                        'spade', 'club', 'diamond', 'heart'], 
                  columns=['Reel1', 'Reel2', 'Reel3', 'Reel4', 'Reel5']) 
df.loc['wild'] = [0,6,6,5,7]
df.loc['scatter'] = [3,3,3,2,3]
df.loc['chip'] = [5,4,4,5,5]
df.loc['man'] = [7,7,5,5,6]
df.loc['car'] = [8,7,5,6,7]
df.loc['casino'] = [8,5,8,8,8]
df.loc['cash'] = [9,6,9,9,9]
df.loc['dice'] = [11,10,10,10,10]
df.loc['spade'] = [12,11,11,11,9]
df.loc['club'] = [12,11,10,13,12]
df.loc['diamond'] = [12,13,13,14,8]
df.loc['heart'] = [12,14,13,14,12]
print("Symbol distribution:")
print(df)

df_total = [] 

for col in df.columns:
    df_total.append(np.sum(df.loc[:,col]))
print("total: ", df_total)
cycle = np.prod(df_total)
print("cycle:", cycle)

Symbol distribution:
         Reel1  Reel2  Reel3  Reel4  Reel5
wild         0      6      6      5      7
scatter      3      3      3      2      3
chip         5      4      4      5      5
man          7      7      5      5      6
car          8      7      5      6      7
casino       8      5      8      8      8
cash         9      6      9      9      9
dice        11     10     10     10     10
spade       12     11     11     11      9
club        12     11     10     13     12
diamond     12     13     13     14      8
heart       12     14     13     14     12
total:  [99, 97, 97, 102, 96]
cycle: 9121159872


In [4]:
pay_table = pd.DataFrame(index=['man','car', 'casino', 
                                'cash', 'dice','spade', 
                                'club', 'diamond', 'heart'], 
                  columns=['5', '4', '3', '2', '1']) 
pay_table.loc['man'] = [150,50,10,2,0]
pay_table.loc['car'] = [100,30,5,0,0]
pay_table.loc['casino'] = [100,25,5,0,0]
pay_table.loc['cash'] = [100,25,5,0,0]
pay_table.loc['dice'] = [75,20,5,0,0]
pay_table.loc['spade'] = [50,10,5,0,0]
pay_table.loc['club'] = [50,10,5,0,0]
pay_table.loc['diamond'] = [50,10,5,0,0]
pay_table.loc['heart'] = [50,10,5,0,0]
print("pay table:\n<==== # of symbols in a pay line ===>")
print(pay_table)
pay_table.columns

pay table:
<==== # of symbols in a pay line ===>
           5   4   3  2  1
man      150  50  10  2  0
car      100  30   5  0  0
casino   100  25   5  0  0
cash     100  25   5  0  0
dice      75  20   5  0  0
spade     50  10   5  0  0
club      50  10   5  0  0
diamond   50  10   5  0  0
heart     50  10   5  0  0


Index(['5', '4', '3', '2', '1'], dtype='object')

### RTP calculation for the base game

In [5]:
hits_table = {}
scatter_temp = []
for count in range(3):
    scatter_temp.append(product_scatter(df, df_total, (reel_count - count)))
print()
hits_table['scatter'] = scatter_temp

for symbol in list(pay_table.index):
    hits_table[symbol] = combine_symbol(df, df_total, pay_table, symbol)

hits_table_base={}
for key,value in hits_table.items():
    temp =  reel_count - len(value)
    if key != 'scatter':
        hits_table_base[key] = list(np.append(hits_table[key], [0]*temp))

print("hits table (for any win) for the basic game in a decreasing order of # occurance, i.e., [5, 4, 3, 2, 1]:")
hits_table_base

An example of hits for any 3 'scatter' symbols:

['~scatter', '~scatter', 'scatter', 'scatter', 'scatter']
['~scatter', 'scatter', '~scatter', 'scatter', 'scatter']
['~scatter', 'scatter', 'scatter', '~scatter', 'scatter']
['~scatter', 'scatter', 'scatter', 'scatter', '~scatter']
['scatter', '~scatter', '~scatter', 'scatter', 'scatter']
['scatter', '~scatter', 'scatter', '~scatter', 'scatter']
['scatter', '~scatter', 'scatter', 'scatter', '~scatter']
['scatter', 'scatter', '~scatter', '~scatter', 'scatter']
['scatter', 'scatter', '~scatter', 'scatter', '~scatter']
['scatter', 'scatter', 'scatter', '~scatter', '~scatter']

hits table (for any win) for the basic game in a decreasing order of # occurance, i.e., [5, 4, 3, 2, 1]:


{'man': [130130, 830830, 8840832, 76632192, 0],
 'car': [176176, 1031888, 9993984, 0, 0],
 'casino': [240240, 1297296, 10526208, 0, 0],
 'cash': [362880, 1814400, 13685760, 0, 0],
 'dice': [718080, 3336960, 23519232, 0, 0],
 'spade': [887808, 4439040, 28631808, 0, 0],
 'club': [1116288, 4523904, 26320896, 0, 0],
 'diamond': [1234620, 6666948, 34517376, 0, 0],
 'heart': [1646160, 6671280, 36334080, 0, 0]}

In [6]:
hits_prob={}
hits_prob_table_base={}
hits_table_total={}
hits_table_total_prob={}

for key in hits_table.keys():
    hits_prob[key] = list((hits_table[key]/cycle))
    hits_table_total[key] = np.sum(hits_table[key])
    hits_table_total_prob[key] = hits_table_total[key]/cycle

for key in hits_table_base.keys():
    hits_prob_table_base[key] = list((hits_table_base[key]/cycle))
    
print("hits probability distribution (for any win) for the base game in a decreasing order of # occurance, i.e., [5, 4, 3, 2, 1]:")
hits_prob_table_base

for key, value in hits_prob_table_base.items():
    print(key, (lambda lst: ['{:0.4e}'.format(x) for x in lst])(value))

hits probability distribution (for any win) for the base game in a decreasing order of # occurance, i.e., [5, 4, 3, 2, 1]:
man ['1.4267e-05', '9.1088e-05', '9.6927e-04', '8.4016e-03', '0.0000e+00']
car ['1.9315e-05', '1.1313e-04', '1.0957e-03', '0.0000e+00', '0.0000e+00']
casino ['2.6339e-05', '1.4223e-04', '1.1540e-03', '0.0000e+00', '0.0000e+00']
cash ['3.9784e-05', '1.9892e-04', '1.5004e-03', '0.0000e+00', '0.0000e+00']
dice ['7.8727e-05', '3.6585e-04', '2.5785e-03', '0.0000e+00', '0.0000e+00']
spade ['9.7335e-05', '4.8667e-04', '3.1391e-03', '0.0000e+00', '0.0000e+00']
club ['1.2238e-04', '4.9598e-04', '2.8857e-03', '0.0000e+00', '0.0000e+00']
diamond ['1.3536e-04', '7.3093e-04', '3.7843e-03', '0.0000e+00', '0.0000e+00']
heart ['1.8048e-04', '7.3141e-04', '3.9835e-03', '0.0000e+00', '0.0000e+00']


In [7]:
base_win={}
base_win_total=[]
for key, value in hits_prob_table_base.items():
    base_win[key] = np.multiply(pay_table.loc[key].values, value)
    base_win_total.append(np.sum(base_win[key]))
print("average win for the base game:", np.sum(base_win_total))

average win for the base game: 0.21871229229562617


### RTP calculation for free games

Let us assume that the number of free spins for the free games is the same as the base game.

If the average win of the base and free game is $B_0$ and $B_1$, respectively, and the base game triggers $n_1$ free games with a trigger probability of $p_1$ per spin, then average number of free games is $n_1p_1$ and the RTP can be calculated as 

\begin{equation*}
RTP = 
\frac{B_0 + B_1n_1p_1}{bet}
\end{equation*}

Now if the free game is able to trigger $n_2$ games with probability of $p_2$, then the average number of free spins can be calculated as 

$$
\begin{equation*}
E(freespins) =
n_1p_1 + n_1p_1(n_2p_2)^2 + n_1p_1(n_2p_2)^3 + ......
 =  n_1p_1\sum_{i=0}^{\infty}{(n_2p_2)^i}
 =  n_1p_1\left(\frac{1}{1-n_2p_2}\right)
\end{equation*}
$$

Hence the final RTP can be found by
$$
\begin{equation*}
RTP = 
\frac{B_0 + B_1n_1p_1\left(\frac{1}{1-n2p2}\right)}{bet}
\end{equation*}
$$

In [8]:
print("total free game hits, i.e., # appearance of min 3 to max 5 'scatter' symbol:\n", 
      "[5 scatters, any 4 scatters, any 3 scatters]: ", hits_table[symbol])
print()
print("scatter hits probability distribution:\n", 
      "[5 scatters, any 4 scatters, any 3 scatters]: ", 
      (lambda lst: ['{:0.4e}'.format(x) for x in lst])(hits_prob['scatter']))

total free game hits, i.e., # appearance of min 3 to max 5 'scatter' symbol:
 [5 scatters, any 4 scatters, any 3 scatters]:  [1646160, 6671280, 36334080]

scatter hits probability distribution:
 [5 scatters, any 4 scatters, any 3 scatters]:  ['4.3159e-06', '2.3833e-04', '5.1982e-03']


In this game, it is assumed that 3, 4 and 5 'scatter' symbols trigger 6, 9 and 15 free games, respectively. First, the expected number of free spins triggered per spin by the base game is the sum of individual triggers, i.e.,
\begin{equation*}
E(freespins) =  
\sum{n.p}
\end{equation*}


In [9]:
free_spins_scatter = np.sum(list(np.multiply(hits_prob['scatter'], [15, 9, 6])))
print("expected free spins by the base game:", free_spins_scatter)

expected free spins by the base game: 0.03339866028827786


In [10]:
B0 = np.sum(base_win_total)
B1 = B0
n1p1 = free_spins_scatter
n2p2= n1p1

RTP_basic = (B0 + B1*n1p1*(1 / (1-n2p2))) / bet

print("RTP for the base game including the free games: ", RTP_basic)

RTP for the base game including the free games:  0.2262693866748153


### RTP calculation for the coid grab feature

The coin values for the 'chip' symbol are considered to be 5000, 3500, 2500, 1500, 1000, 500 and 100 for the following payout calculations. It is to be noted that, if more features are added to the game, then the coin values may be fine tuned in order to obtain the desired payout percentage. 

In [11]:
df_chip = pd.DataFrame(index=['5000', '3500', '2500',
                             '1500', '1000', '500', '100'], 
                  columns=['Reel1', 'Reel2', 'Reel3', 'Reel4', 'Reel5']) 
df_chip.loc['5000'] =[4]*reel_count
df_chip.loc['3500'] = [6]*reel_count
df_chip.loc['2500'] = [8]*reel_count
df_chip.loc['1500'] = [10]*reel_count
df_chip.loc['1000'] = [12]*reel_count
df_chip.loc['500'] = [14]*reel_count
df_chip.loc['100'] = [16]*reel_count
print("'chip' distribution for different coin values:\n")
print(df_chip)

df_chip_total = [] 

for col in df_chip.columns:
    df_chip_total.append(np.sum(df_chip.loc[:,col]))
print("total 'chip' count: ", [df_chip_total[0]]*5)
chip_cycle = np.prod(df_chip_total)
print("chip cycle: ", chip_cycle)

'chip' distribution for different coin values:

      Reel1  Reel2  Reel3  Reel4  Reel5
5000      4      4      4      4      4
3500      6      6      6      6      6
2500      8      8      8      8      8
1500     10     10     10     10     10
1000     12     12     12     12     12
500      14     14     14     14     14
100      16     16     16     16     16
total 'chip' count:  [70, 70, 70, 70, 70]
chip cycle:  1680700000


In [12]:
chip_hits_table, chip_hits_length = combine_chip(6)
print("Coin grab feature is triggered if minimum of 6 'chip' symbols appear on the screen.")
print("An example of how 6 'chip' symbols may appear on the screen and " 
      "their corresponding probability is shown below.\n")
print("There are {} ways 6 chips can be combined:".format(chip_hits_length))
print()
for key, value in chip_prob_dist(df, df_total, 6).items():
    print(chip_hits_table[key], ": ", (lambda lst: ['{:0.4e}'.format(x) for x in lst])(value))

Coin grab feature is triggered if minimum of 6 'chip' symbols appear on the screen.
An example of how 6 'chip' symbols may appear on the screen and their corresponding probability is shown below.

There are 135 ways 6 chips can be combined:

(3, 3, 0, 0, 0) :  ['3.8253e-04', '1.6278e-04', '8.7629e-01', '8.5294e-01', '8.4375e-01']
(3, 0, 3, 0, 0) :  ['3.8253e-04', '8.7629e-01', '1.6278e-04', '8.5294e-01', '8.4375e-01']
(3, 0, 0, 3, 0) :  ['3.8253e-04', '8.7629e-01', '8.7629e-01', '3.4945e-04', '8.4375e-01']
(3, 0, 0, 0, 3) :  ['3.8253e-04', '8.7629e-01', '8.7629e-01', '8.5294e-01', '4.1993e-04']
(0, 3, 3, 0, 0) :  ['8.4848e-01', '1.6278e-04', '1.6278e-04', '8.5294e-01', '8.4375e-01']
(0, 3, 0, 3, 0) :  ['8.4848e-01', '1.6278e-04', '8.7629e-01', '3.4945e-04', '8.4375e-01']
(0, 3, 0, 0, 3) :  ['8.4848e-01', '1.6278e-04', '8.7629e-01', '8.5294e-01', '4.1993e-04']
(0, 0, 3, 3, 0) :  ['8.4848e-01', '8.7629e-01', '1.6278e-04', '3.4945e-04', '8.4375e-01']
(0, 0, 3, 0, 3) :  ['8.4848e-01', '8.7

In [13]:
df_chip_ix_combo, df_chip_ix = combine_coin_values(df_chip, 2)
print("chip counts can be 3, 2, 1 or 0 in a reel.")
print("As an example, # of ways any 2 coin values in a reel can be combined:\n")
for key,value in df_chip_ix_combo.items():
    print(key, value)

chip counts can be 3, 2, 1 or 0 in a reel.
As an example, # of ways any 2 coin values in a reel can be combined:

0 [('5000', '5000')]
1 [('5000', '3500'), ('3500', '5000')]
2 [('5000', '2500'), ('2500', '5000')]
3 [('5000', '1500'), ('1500', '5000')]
4 [('5000', '1000'), ('1000', '5000')]
5 [('5000', '500'), ('500', '5000')]
6 [('5000', '100'), ('100', '5000')]
7 [('3500', '3500')]
8 [('3500', '2500'), ('2500', '3500')]
9 [('3500', '1500'), ('1500', '3500')]
10 [('3500', '1000'), ('1000', '3500')]
11 [('3500', '500'), ('500', '3500')]
12 [('3500', '100'), ('100', '3500')]
13 [('2500', '2500')]
14 [('2500', '1500'), ('1500', '2500')]
15 [('2500', '1000'), ('1000', '2500')]
16 [('2500', '500'), ('500', '2500')]
17 [('2500', '100'), ('100', '2500')]
18 [('1500', '1500')]
19 [('1500', '1000'), ('1000', '1500')]
20 [('1500', '500'), ('500', '1500')]
21 [('1500', '100'), ('100', '1500')]
22 [('1000', '1000')]
23 [('1000', '500'), ('500', '1000')]
24 [('1000', '100'), ('100', '1000')]
25 [('

In [14]:
# 'chip_prob_dist_combined' returns the combined probability of each of the possible 
#  combinations that may appear on the screen for a given number of chip counts
#  (for a total of min=6 to max=15 chip 'symbols') in a reel. 

chip_payout_sum = 0
chip_prob={}
for i in range(6,16):
    chip_prob_combined = chip_prob_dist_combined(df, df_total, i)
    prob_combined=0
    for key, value in chip_prob_combined.items():
        prob_combined += value
    chip_prob[i] = prob_combined
print("Chip probability distribution (for a total occurance of min=6 to max=15 symbols):\n")
for key, value in chip_prob.items():
    print(key, ": ", value)

Chip probability distribution (for a total occurance of min=6 to max=15 symbols):

6 :  0.000115419464628714
7 :  1.0079900157390907e-05
8 :  7.02394550403594e-07
9 :  3.8977540953559674e-08
10 :  1.705177267264151e-09
11 :  5.763671236316754e-11
12 :  1.4536958770515434e-12
13 :  2.575486193399213e-14
14 :  2.855770663993313e-16
15 :  1.487380554163184e-18


In [15]:
# 'chip_payout_combined' returns average win for a given number of 
# 'chip' symbol occurances (from min=6 to max=15) taking into account all
#  possible coin value combinations calculated above.

chip_win_tmp = 0
chip_prob_tmp = 0
for i in range(6,16):
    chip_prob_combined = chip_prob_dist_combined(df, df_total, i)
    chip_win_tmp += chip_payout_combined(df_chip, df_chip_total, chip_prob_combined, i)
    chip_prob_tmp += chip_prob[i]

In this game, 3 free spins are awarded as coin grab feature is triggered. If one or more additional 'chip' appear in any position these are also held and the number of free spins remaining is reset to 3. If the average win for the coin grab feature and free game during the coin grab feature are $B_0$ and $B_1$, respectively, then the RTP for the coin grab feature can be calculated as follows (same as what has been explained previously).

In [16]:
B0 = chip_win_tmp
B1 = B0
p1 = chip_prob_tmp
n1p1 = 3*p1
n2p2 = n1p1

RTP_coin_grabber = (B0 + B1*n1p1*(1 / (1-n2p2))) / bet
RTP_coin_grabber

0.6612458042569189

In [17]:
Final_RTP = RTP_basic + RTP_coin_grabber
print("The final payout percentage: {}%".format(np.round(Final_RTP*100,4)))

The final payout percentage: 88.7515%
